In [138]:
import numpy as np
import matplotlib.pyplot
import mnist
import scipy
import scipy.sparse.linalg
import scipy.misc
from scipy.sparse import csr_matrix
from scipy.sparse import identity
from numpy import linalg as LA
import pdb

In [139]:
trainX, trainY = mnist.load_mnist("training", None, './MNIST');
testX, testY = mnist.load_mnist("testing", None, './MNIST');

print("MNIST loaded")

MNIST loaded


In [140]:
class PCA:
    
    def __init__(self):
        self.sigma = 0;
        
    def load_train(self, trainX):
        s = trainX.shape;
        self.d = s[1] * s[2];
        self.N = s[0];
        self.X = np.reshape(trainX, (self.N, self.d));
        print("PCA : Training data loaded...")
        
    def compSigma(self):
        self.Sigma = self.X.transpose().dot(self.X);
        self.w, self.v = LA.eig(self.Sigma);
        print("PCA : Sigma Computed...")
    
    def decomposeInput(self, k):
        weights = np.zeros((self.N, k));
        for i in range(0, k):
            weights[:, i] = np.reshape(self.X.dot(np.reshape(self.v[:,i], (784,1))), self.N);
        print("PCA : Data decomposed to " + str(k) + " dimensions...")
        return weights
    
    def decompose(self, procdata, k):
        weights = np.zeros((procdata.shape[0], k));
        for i in range(0, k):
            weights[:, i] = np.reshape(procdata.dot(np.reshape(self.v[:,i], (784,1))), procdata.shape[0]);
        print("PCA : Input data decomposed to " + str(k) + " dimensions...")
        return weights

In [141]:
testX = np.reshape(testX, (testX.shape[0], testX.shape[1] * testX.shape[2]));

model = PCA();
model.load_train(trainX);
model.compSigma();
pca50Xtr = model.decomposeInput(50);
pca50Xte = model.decompose(testX, 50);

PCA : Training data loaded...
PCA : Sigma Computed...
PCA : Data decomposed to 50 dimensions...
PCA : Input data decomposed to 50 dimensions...


In [230]:
class neuralnet:
    def __init__(self, lrnRate, connect, nNeurons_FC, batch_size):
        self.connect = connect;
        self.nNeurons_FC = nNeurons_FC;
        self.nLayers = len(connect);
        self.lrnRate = lrnRate;
        self.batchSize = batch_size
        print("NN : Running Neural Net ", [connect[i] for i in range(self.nLayers)])
    
    def load_train(self, trainX, trainY):
        s = trainX.shape;
        self.d = s[1];
        self.N = s[0];
        self.K = np.max(trainY)+1
        #self.trainX = np.insert(trainX, 0, 1, axis=1);
        self.trainX = trainX
        
        self.trainY1 = trainY;
        self.trainY = np.zeros((self.N, self.K),dtype=np.float)
        for i in range(0,self.K):
            temp = np.reshape(np.array(trainY == i, dtype=float), (self.N, 1));
            self.trainY[:,i] = np.asmatrix(np.reshape(temp, self.N))
            
        print("NN : Training data loaded...")
        
    def load_batch(self):
        #print (str(self.batchFrom) + ' ' + str(self.batchTo))
        self.Y1 = self.trainY1[self.perm[self.batchFrom:self.batchTo]]
        self.X = self.trainX[self.perm[self.batchFrom:self.batchTo], :]
        #self.X = np.insert(self.X, 0, 1, axis=1);
        self.Y = self.trainY[self.perm[self.batchFrom:self.batchTo], :]
        self.batchFrom += self.batchSize
        self.batchTo += self.batchSize
        self.batchFrom = (self.batchFrom - self.N) if self.batchFrom >= self.N else self.batchFrom
        self.batchTo = (self.batchTo - self.N) if self.batchTo > self.N else self.batchTo
        if self.batchFrom > self.batchTo or self.batchFrom==0:
            self.newBatch = True;
            self.batchFrom += self.batchSize
            self.batchTo += self.batchSize
            self.batchFrom = (self.batchFrom - self.N) if self.batchFrom >= self.N else self.batchFrom
            self.batchTo = (self.batchTo - self.N) if self.batchTo > self.N else self.batchTo
            self.iterAll = self.iterAll + 1;
            self.perm = np.random.permutation(self.N)
        else:
            self.newBatch = False;
    
    def init_net(self):
        if self.connect[0] is "FC":
            self.layers = [np.zeros((self.nNeurons_FC[0], self.d))];
            self.bias = [np.ones((1, self.nNeurons_FC[0]))];
        else:
            print("Currently only support when the first layer is FC")
            
        for i in range(1,self.nLayers):
            if self.connect[i] is "FC":
                self.layers.append(np.zeros((self.nNeurons_FC[i], self.nNeurons_FC[i-1])));
                self.bias.append(np.ones((1, self.nNeurons_FC[i])));
            else:
                self.layers.append(np.zeros(0));
                self.bias.append(np.zeros(0));
        
        self.layervals = []
        for i in range(self.nLayers):
            if self.connect[i] is "LOSS":
                self.layervals.append(np.zeros(1));
            else:
                self.layervals.append(np.zeros((self.batchSize, self.nNeurons_FC[i])));
                
        self.gradients = [];
        for i in range(self.nLayers):
            self.gradients.append(np.zeros((self.batchSize, self.nNeurons_FC[i])));
        
        self.iter = 0;
        self.iterAll = 0;
        self.prevLoss = 999999;
        self.weightDist = 99999;
        self.batchFrom = 0;
        self.batchTo = self.batchSize;
        self.perm = range(0, self.N)
        print("NN : Neural Net initialized...")
    
    def forward(self):
        curval = self.X;
        for i in range(self.nLayers):
            if self.connect[i] is "FC":
                curval = curval.dot(self.layers[i].transpose()) + np.tile(self.bias[i],(self.batchSize, 1));
                self.layervals[i] = curval
            elif self.connect[i] is "TANH":
                
                curval = np.tanh(curval);
                self.layervals[i] = curval
                #pdb.set_trace()
            elif self.connect[i] is "LOSS":

                temp = (curval - self.Y)*(curval - self.Y);
                temp = np.sum(temp, axis = 1);
                self.layervals[i] = temp
                self.loss = np.sum(temp);
        
        pred = np.argmax(curval, axis = 1);
        correct = np.sum(np.float16(pred == self.Y1))
        self.loss01 = correct / np.float16(self.batchSize);
        print("NN : forwarded " + str(self.nLayers) + " layers...")
        print("NN : train loss : " + str(self.loss));
        print("NN : train 0/1 loss : " + str(self.loss01));
    
    def compGradLoss(self, i):
        self.gradients[i] = 2 * (self.layervals[i-1] - self.Y) / self.batchSize;
        
    def compGradFC(self, i):
        # Forward using function : Y = X * W + b;
        # Backward using function : dL / dX = (dL / dY) * (dY / dX) = (dL / dY) * W.trans(); 
        pdb.set_trace()
        self.gradients[i] = self.gradients[i+1].dot(self.layers[i]);
        if i==0:
            weightGrad = self.gradients[i+1].transpose().dot(self.X);
        else:
            weightGrad = self.gradients[i+1].transpose().dot(self.layervals[i-1]);
        
        self.layers[i] = self.layers[i] - self.lrnRate * weightGrad;
        
        biasGrad = self.gradients[i+1].transpose().dot(np.ones((self.batchSize, 1)));
        self.bias[i] = self.bias[i] - self.lrnRate * biasGrad.transpose();
        
    def compGradTanh(self, i):
        self.gradients[i] = self.gradients[i+1] * (1 - np.tanh(self.layervals[i])*np.tanh(self.layervals[i]));
        
    def forward_all(self, inX, inY):
        curval = inX;
        self.testN = inX.shape[0];
        
        inY1 = inY;
        inY = np.zeros((inY.shape[0], 10),dtype=np.float)
        for i in range(0,10):
            temp = np.reshape(np.array(testY1 == i, dtype=float), (inY.shape[0], 1));
            inY[:,i] = np.asmatrix(np.reshape(temp, inY.shape[0]))
        
        
        layervals = []
        for i in range(self.nLayers):
            if self.connect[i] is "LOSS":
                layervals.append(np.zeros(1));
            else:
                layervals.append(np.zeros((self.testN, self.nNeurons_FC[i])));
        
        
        for i in range(self.nLayers):
            if self.connect[i] is "FC":
                curval = curval.dot(self.layers[i].transpose()) + np.tile(self.bias[i],(self.testN, 1));
                layervals[i] = curval
            elif self.connect[i] is "TANH":
                
                curval = np.tanh(curval);
                layervals[i] = curval
                #pdb.set_trace()
            elif self.connect[i] is "LOSS":
                temp = (curval - inY)*(curval - inY);
                
                self.lossTest = np.sum(temp, axis = 1);
                layervals[i] = self.lossTest
                self.lossTest = np.sum(self.lossTest) / 10 / self.testN;
        
        pred = np.argmax(curval, axis = 1);
        if self.iter == 20:
            pdb.set_trace()
        correct = np.sum(np.float16(pred == inY1))
        self.loss01 = correct / np.float16(self.testN);
            
        print("NN : test loss : " + str(self.lossTest));
        print("NN : test 0/1 loss : " + str(self.loss01));
    

In [233]:
connect = ["FC", "TANH", "FC", "LOSS"];
nNeurons_FC = [500, 500, 10, 1];

NN = neuralnet(0.0001, connect, nNeurons_FC, 10);

NN.load_train(pca50Xtr, trainY);
NN.init_net();

NN.iter = 100
while NN.iter>0:
    NN.load_batch();
    NN.forward();
    NN.compGradLoss(3)
    NN.compGradFC(2)
    pdb.set_trace()
    NN.compGradTanh(1)
    NN.compGradFC(0)
    pdb.set_trace()
    NN.forward();
    NN.forward_all(pca50Xte, testY);
    NN.iter -= 1;
    #pdb.set_trace()
    print("NN : Loss " + str(NN.loss))
    print("\n\n")


('NN : Running Neural Net ', ['FC', 'TANH', 'FC', 'LOSS'])
NN : Training data loaded...
NN : Neural Net initialized...
NN : forwarded 4 layers...
NN : train loss : 90.0
NN : train 0/1 loss : 0.099976
> <ipython-input-230-7a9aa83d82fd>(114)compGradFC()
-> self.gradients[i] = self.gradients[i+1].dot(self.layers[i]);
(Pdb) continue
> <ipython-input-233-c69f53f1beb2>(16)<module>()
-> NN.compGradTanh(1)
(Pdb) continue
> <ipython-input-230-7a9aa83d82fd>(114)compGradFC()
-> self.gradients[i] = self.gradients[i+1].dot(self.layers[i]);
(Pdb) continue
> <ipython-input-233-c69f53f1beb2>(19)<module>()
-> NN.forward();
(Pdb) continue
NN : forwarded 4 layers...
NN : train loss : 80.755160892
NN : train 0/1 loss : 0.30005
NN : test loss : 0.808448859675
NN : test 0/1 loss : 0.11353
NN : Loss 80.755160892



NN : forwarded 4 layers...
NN : train loss : 80.8715660237
NN : train 0/1 loss : 0.099976
> <ipython-input-230-7a9aa83d82fd>(114)compGradFC()
-> self.gradients[i] = self.gradients[i+1].dot(self.la

KeyboardInterrupt: 

In [204]:
NN.layervals[1].shape

(10, 500)